In [1]:
# Utils
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import time
#import _pickle as pickle
import pickle
import re, sys, unidecode
#import unidecode


# Representation
import nltk
from nltk.corpus import stopwords
import scikitplot.plotters as skplt

import wordbatch
from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL

from tensorflow.contrib.learn import DNNClassifier
from tensorflow.contrib.learn import DNNEstimator

#from tecnosmartlib import DataObject

/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so | _pywrap_tensorflow_internal
/root/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/lite/toco/python/_tensorflow_wrap_toco.so | _tensorflow_wrap_toco


In [ ]:
#plt.style.use('fivethirtyeight')


nltk.download('stopwords')
nltk.download('punkt')

# Load Data

In [2]:
dfNoticias = pd.read_pickle("dfNoticias")

# pre procesing

Cleaning data, select clases

In [3]:
print('Selecting useful information...', end='')
_map = {}
_map["Cuerpo"] = []
_map["Seccion"] = []
_map["Seccion2"] = []
_map["Seccion3"] = []

count = 0
for index, row in dfNoticias.iterrows():
    if (row["Seccion_1"] != None and row["Cuerpo"] != None):
        _map["Cuerpo"].append(row["Cuerpo"]) 
        _map["Seccion"].append(row["Seccion_1"])
        _map["Seccion2"].append(row["Seccion_2"])
        _map["Seccion3"].append(row["Seccion_3"])
        if row["Seccion_1"] == row["Seccion_2"] and row["Seccion_2"] == row["Seccion_3"]:
            count += 1
        if row["Seccion_1"] == row["Seccion_2"] and row["Seccion_3"] == None:
            count += 1
        if row["Seccion_1"] == row["Seccion_3"] and row["Seccion_2"] == None:  
            count += 1 
        if row["Seccion_2"] == None and row["Seccion_3"] == None:
            count += 1      
print('done.')



Selecting useful information...done.


In [4]:
print( len(_map['Seccion2']) )
print(count)

250339
220068


In [5]:
df = pd.DataFrame(_map)
numberOfDuplicates = df.shape[0]
print('Number of data before dropping duplicates: {}'.format(df.shape[0]))
df = df.drop_duplicates(inplace= False)
df.reset_index(drop=True, inplace= True)
numberOfDuplicates -= df.shape[0]
print('Number of data after dropping duplicates : {}'.format(df.shape[0]))
print('\nNumber of duplicated data : {}'.format(numberOfDuplicates))

#df.head()

Number of data before dropping duplicates: 250339
Number of data after dropping duplicates : 238931

Number of duplicated data : 11408


In [6]:
X_untransformed = df['Cuerpo'].reset_index(drop=True)
y_untransformed = df['Seccion'].reset_index(drop=True)

assert X_untransformed.shape[0] == y_untransformed.shape[0], 'X and y dimenssion must be the same.'
print('Number of data: {}'.format(X_untransformed.shape[0]))

Number of data: 238931


In [7]:
classes = df.groupby('Seccion')['Cuerpo'].nunique()

print(classes)
nClasses = classes.shape[0]

#print('\nNumber of classes: {}'.format(nClasses))

Seccion
Corporativo        82
Cultura          3128
Deportes        76458
Economía         6750
Entretención    29438
Mundo           31664
País            75180
Sociedad         9079
Tecnología       7130
Name: Cuerpo, dtype: int64


# Feature Extraccion

### Data representation

TFID calculation

In [8]:

start = time.time()
print("start")


spanish_stopwords = stopwords.words('spanish')
def normalize_text(text):
    return u" ".join([x for x in [y for y in text.lower().strip().split(" ")] 
                      if len(x) > 1 and x not in spanish_stopwords])

X_untransformed = X_untransformed
n_docs = X_untransformed.shape[0]
n_cpu = 20

batch_size = int(n_docs/n_cpu)

wb = wordbatch.WordBatch(normalize_text, 
                         extractor=(WordBag, {"hash_ngrams": 1, "hash_ngrams_weights": [1.0, 1.0],
                                              "hash_size": 2**28, "norm": "l2", "tf": 1.0,
                                              "idf": 1.0}), procs=n_cpu, n_words=1000, minibatch_size=batch_size)
wb.dictionary_freeze = True
word_comment = wb.fit_transform(list(X_untransformed),reset= False)
X_transformed = word_comment[:, np.array(np.clip(word_comment.getnnz(axis=0) - 1, 0, 1), dtype = bool)]

end = time.time()
print("TFIDF end time :" + str(end - start) )


X = X_transformed
print('Number of features: {}'.format(X.shape[1]))

start
Normalize text
Extract wordbags
TFIDF end time :101.38772940635681
Number of features: 1000


### Output

In [9]:
class DataObject():
    def __init__(self, x, y):
        self.X = x
        self.Y = y

        try:
            self.Y.reset_index(drop=True, inplace=True)
        except:
            pass

        self.X_train = self.X
        self.Y_train = self.Y

        self.X_test  = None
        self.Y_test  = None

        self.X_test_dense = None

        self.k = 0
    def next_train_batch(self, batch_size):
        indexes = np.random.choice(self.X_train.shape[0], size=batch_size, replace=False)
        return self.X_train[indexes], self.Y_train.reindex(indexes)

    def set_train_test(self,train_fraction = None, test_fraction = None, dense=True):
        # Tests for create subsets
        if train_fraction == None and test_fraction == None:
            train_fraction = 0.8
        elif train_fraction == None:
            assert 0 <= test_fraction and test_fraction < 1, 'Test fraction must be in [0,1)'
            train_fraction = 1 - test_fraction
        else:
            assert 0 < train_fraction and train_fraction <= 1, 'Train fraction must be in (0,1]'

        train_indices = np.random.choice(self.X.shape[0], round(train_fraction*self.X.shape[0]), replace=False)
        test_indices = np.array(list(set(range(self.X.shape[0])) - set(train_indices)))

        self.X_train = self.X[train_indices]
        self.Y_train = self.Y.reindex(train_indices)
        self.Y_train = self.Y_train.reset_index()

        self.X_test = self.X[test_indices]
        self.Y_test = self.Y.reindex(test_indices)
        self.Y_test = self.Y_test.reset_index()
        if dense:
            self.X_test_dense = self.X_test.todense()
        return True


Y = y_untransformed
Y = Y.reset_index(drop=True)

def remove_accents(a):
    return unidecode.unidecode(a)

data = DataObject(X,Y.apply(remove_accents))

labels = [remove_accents(x) for x in list(data.Y.unique())]
#print(labels)

data.set_train_test(0.8)
#print(data.Y_train.reset_index(drop=True))

True

# Model

### DNN graph generation

In [27]:
tf.set_random_seed(42)

feature_columns = [tf.contrib.layers.real_valued_column('x', dimension=1000)]

classifier = DNNClassifier(
                           n_classes=nClasses, label_keys=labels, feature_columns=feature_columns,
                           hidden_units=[2000, 1000, 100],
                           dropout=0.8,
                           weight_column_name='class_weights',
                           #model_dir = './model-parallel/seccion_dnn',
                           model_dir = './tmp',    
                           config = tf.contrib.learn.RunConfig(save_checkpoints_steps = 500,
                           save_checkpoints_secs = None)                           
                          )

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f891fa88748>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 500, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './tmp'}


### functions graph tensorflow

In [11]:
epochs = 2

# Define the test inputs
def get_train_inputs():
    print("Paso por aca.........................")
    X_ = data.X_train
    Y_ = data.Y_train
    X_ = X_.todense()
    x = X_
    Y_ = Y_['Seccion'].reset_index(drop=True)
#     y = y.apply(remove_accents)
#     y = y.values

    y = Y_.values
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}
#     print(sizes)

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print('weights shape: {}'.format(weights.shape))
    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=True, batch_size=500, num_epochs=epochs)
    return dataset

def get_test_inputs():
    X_ = data.X_test_dense
    Y_ = data.Y_test
#     X_ = X_.todense()
    x = X_
    Y_ = Y_['Seccion'].reset_index(drop=True)
    y = Y_.values
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}
#     print(sizes)

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print('weights shape: {}'.format(weights.shape))
    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=False)
    return dataset


In [24]:
estimator = DNNEstimator(
    head=tf.contrib.estimator.multi_label_head(n_classes=9),
    feature_columns=feature_columns,
    #feature_columns=[tf.feature_column.numeric_column("", shape=[1000, 1])],
    hidden_units=[2000, 1000, 100],
    optimizer=tf.train.ProximalAdagradOptimizer(
      learning_rate=0.1,
      l1_regularization_strength=0.001
    ))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f891f88b320>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpgijwm4n0'}


In [29]:
# Input builders

estimator.fit(input_fn=get_train_inputs)

Paso por aca.........................
weights shape: (191145, 1)


TypeError: 'function' object is not iterable

# Training



In [28]:
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn= get_test_inputs(),
    every_n_steps=500,
    early_stopping_metric="accuracy",
    early_stopping_rounds = 5000)

classifier.fit(input_fn=get_train_inputs(), monitors=[validation_monitor], steps=epochs, max_steps=None)

weights shape: (47786, 1)
Paso por aca.........................
weights shape: (191145, 1)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt-2
INFO:tensorflow:Saving checkpoints for 3 into ./tmp/model.ckpt.
INFO:tensorflow:loss = 0.9336558, step = 3
INFO:tensorflow:Saving checkpoints for 4 into ./tmp/model.ckpt.
INFO:tensorflow:Loss for final step: 0.97240984.


DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f891fabecf8>, 'hidden_units': [2000, 1000, 100], 'feature_columns': (_RealValuedColumn(column_name='x', dimension=1000, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x7f8a08c0d2f0>, 'dropout': 0.8, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

# Evaluation performance


In [ ]:
def input_fn_evaluate():
    dataset = {'x': tf.constant(data.X_test_dense)}
    
    return dataset

pred_test = classifier.predict_classes(input_fn=input_fn_evaluate)

# print(pred_test)

y_test_hat = np.asarray([x.decode('UTF-8') for x in list(pred_test)])
y_test_hat = y_test_hat.astype(str)
y_test = data.Y_test['Seccion'].values
y_test = y_test.astype(str)
print('Test shape: {}\nReal shape: {}'.format(y_test_hat.shape, y_test.shape))

from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)
print('Accuracy in test: {}'.format(acc))





In [ ]:
predict_proba = classifier.predict_proba(input_fn=input_fn_evaluate,as_iterable=False)
np.sum(predict_proba, axis=1)

In [ ]:

ax = skplt.plot_confusion_matrix(y_test, y_test_hat,normalize=True)
# plt.xticks(rotation=45)

for tick in ax.xaxis.get_minor_ticks():
    tick.tick1line.set_markersize(0)
    tick.tick2line.set_markersize(0)
    tick.label1.set_horizontalalignment('center')

plt2.xticks(rotation=70)